In [1]:
import pyomo.environ as pyo
import pandas as pd
from pathlib import Path
import preprocess_data as ppd
from itertools import zip_longest

known_x_sols = [[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]]

## Data

In [2]:
data_path = Path("../data/")
output_path = Path("../output/")
df, adjacent_matrix = ppd.get_df_adj(data_path, 2021)
model = pyo.AbstractModel()

## Defs

In [3]:
def param_adjacent(m, i, j):
    return int(j in adjacent_matrix[i])

def param_pop(m, i):
    return df["population"][i - 1]

def con_a(m, i):
    return sum((m.a[i, j] * m.x[j]) for j in m.J) >= m.y[i]

def con_x(m):
    return sum(m.x[j] for j in m.J) <= m.k

def obj_sum(m):
    return pyo.summation(m.p, m.y)

## Constraints

In [4]:
# value of n (number of counties)
model.n = 88

# TOSET limit on number of pirincipal places of buisnesses opened (init to 5)
model.k = 15

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

model.p = pyo.Param(model.I, initialize=param_pop)  # population of county i

# model.a = pyo.Set(model.I, model.J, within=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent
model.a = pyo.Param(model.I, model.J, domain=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent

model.x = pyo.Var(model.J, domain=pyo.Binary)  # 1 if principal place of business is opened in county j
model.y = pyo.Var(model.I, domain=pyo.Binary)  # 1 if county i is covered

model.obj = pyo.Objective(rule=obj_sum, sense=pyo.maximize)

model.a_constraint = pyo.Constraint(model.I, rule=con_a)
model.x_constraint = pyo.Constraint(rule=con_x)

## Solve

In [5]:
# opt = pyo.SolverFactory("ipopt", executable="/home/adb/anaconda3/bin/ipopt")
opt = pyo.SolverFactory("glpk", executable="/home/adb/anaconda3/bin/glpsol")
# opt = pyo.SolverFactory("glpk", executable="/usr/local/Caskroom/miniconda/base/bin/glpsol")

In [6]:
instance1 = model.create_instance()
results1 = opt.solve(instance1).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 11769923.0
  Upper bound: 11769923.0
  Number of objectives: 1
  Number of constraints: 90
  Number of variables: 177
  Number of nonzeros: 729
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 31
      Number of created subproblems: 31
  Error rc: 0
  Time: 0.01277923583984375
# ----------------------------------------------------------
#   Solution Information
# --------------------

In [7]:
sol_dict1 = instance1.x.get_values()
sol1 = [k for k, v in sol_dict1.items() if v == 1]
not_sol1 = set([i for i in range(1, model.n+1)]) - set(sol1)
sol1_camm = ppd.county_ids_to_camm_ids(df, sol1)

print(f"Solution 1: \n{sol1}")
print(f"Solution 1 with Camm18 indexing method: \n{sol1_camm}")

Solution 1: 
[3, 5, 7, 8, 9, 28, 35, 40, 45, 49, 51, 72, 75, 76, 81]
Solution 1 with Camm18 indexing method: 
[4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]


## Re-solve for 2nd solution

In [8]:
model = model.clone()

def con_x_sol2(m):
    return sum(m.x[i] for i in sol1) - sum(m.x[j] for j in not_sol1) <= len(sol1) - 1
model.sol2_constraint = pyo.Constraint(rule=con_x_sol2)

instance2 = model.create_instance()
results2 = opt.solve(instance2)

In [9]:
sol_dict2 = instance2.x.get_values()
sol2 = [k for k, v in sol_dict2.items() if v == 1]
not_sol2 = set([i for i in range(1, model.n+1)]) - set(sol2)
sol2_camm = ppd.county_ids_to_camm_ids(df, sol2)

print(f"Solution 2: \n{sol2}")
print(f"Solution 2 with Camm18 indexing method: \n{sol2_camm}")

Solution 2: 
[3, 5, 7, 8, 9, 28, 35, 40, 45, 49, 51, 69, 72, 75, 76]
Solution 2 with Camm18 indexing method: 
[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]


## Re-solve for 3rd solution

In [10]:
model = model.clone()

def con_x_sol3(m):
    return sum(m.x[i] for i in sol2) - sum(m.x[j] for j in not_sol2) <= len(sol2) - 1
model.sol3_constraint = pyo.Constraint(rule=con_x_sol3)

instance3 = model.create_instance()
results2 = opt.solve(instance3)

In [11]:
sol_dict3 = instance3.x.get_values()
sol3 = [k for k, v in sol_dict3.items() if v == 1]
sol3 = set([i for i in range(1, model.n+1)]) - set(sol3)
sol3_camm = ppd.county_ids_to_camm_ids(df, sol3)

# [3, 5, 7, 8, 9, 28, 35, 40, 45, 49, 51, 72, 75, 76, 81]
# [3, 5, 7, 8, 9, 28, 35, 40, 45, 49, 51, 69, 72, 75, 76]
print(f"Solution 3: \n{sol3}")
print(f"Solution 3 with Camm18 indexing method: \n{sol3_camm}")

Solution 3: 
{1, 2, 4, 6, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 46, 47, 48, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 73, 74, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88}
Solution 3 with Camm18 indexing method: 
[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 34, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 88]


In [12]:
instance3.sol2_constraint.pprint()

sol2_constraint : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   : Upper : Active
    None :  -Inf : x[3] + x[5] + x[7] + x[8] + x[9] + x[28] + x[35] + x[40] + x[45] + x[49] + x[51] + x[72] + x[75] + x[76] + x[81] - (x[1] + x[2] + x[4] + x[6] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + 

In [13]:
instance3.sol3_constraint.pprint()

sol3_constraint : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   : Upper : Active
    None :  -Inf : x[3] + x[5] + x[7] + x[8] + x[9] + x[28] + x[35] + x[40] + x[45] + x[49] + x[51] + x[69] + x[72] + x[75] + x[76] - (x[1] + x[2] + x[4] + x[6] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + 

## Verify

In [16]:
pd.DataFrame(zip_longest(sol1_camm, known_x_sols[0], sol2_camm, known_x_sols[1], sol3_camm, known_x_sols[2]), columns=["sol1", "camm_sol1", "sol2", "camm_sol2", "sol3", "camm_sol3"])

print(f"Solution 1 \nResult:{sol1_camm} \nCamm18:{known_x_sols[1]} \n ----")
print(f"Solution 2 \nResult:{sol2_camm} \nCamm18:{known_x_sols[2]} \n ----")
print(f"Solution 3 \nResult:{sol3_camm} \nCamm18:{known_x_sols[0]} \n ----")

Solution 1 
Result:[4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
Camm18:[4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
 ----
Solution 2 
Result:[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
Camm18:[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
 ----
Solution 3 
Result:[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 34, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 88] 
Camm18:[8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87] 
 ----


## Save solution for visualization

In [15]:
sol_save = pd.Series(sol2)
sol_save.to_csv(output_path / "solution_model_1_static.csv", index=False, header=['county_id'])